In [1]:
import pandas as pd
import numpy as np
import Clients.OddsApiClient as oddsApi
import sys
from Models.H2hEvent import H2hEvent, H2hBookmaker 
from Transformers.H2hEventTransformer import oddsApiEventsToH2hEvent
import importlib


In [2]:
from datetime import datetime, timedelta, timezone

current_utc_time = datetime.now(timezone.utc)
future_time = current_utc_time + timedelta(hours=12)
iso_formatted_time_with_z = future_time.isoformat(timespec="seconds").replace(
    "+00:00", "Z"
)
current_formatted_time = current_utc_time.isoformat(timespec="seconds").replace(
    "+00:00", "Z"
)
print(iso_formatted_time_with_z)

2025-08-03T01:19:01Z


In [3]:
importlib.reload(oddsApi)

sports = oddsApi.ALL_SOCCER_SPORTS + oddsApi.All_OTHER_SPORTS

soccer_events = oddsApi.getEventsForMultipleSports(
    sports=sports[0:4], regions=[oddsApi.Regions.UK], markets=[oddsApi.Markets.H2H], commenceTimeFrom=current_formatted_time, commenceTimeTo=iso_formatted_time_with_z)

Number of events: 0
[]
Remaining requests 15795.0
Used requests 4205
Number of events: 3
[{'id': 'c4f39d8e047ef5276d4c80fcd9c4628b', 'sport_key': 'soccer_austria_bundesliga', 'sport_title': 'Austrian Football Bundesliga', 'commence_time': '2025-08-02T15:00:00Z', 'home_team': 'WSG Tirol', 'away_team': 'Hartberg', 'bookmakers': [{'key': 'betfair_sb_uk', 'title': 'Betfair Sportsbook', 'last_update': '2025-08-02T13:18:13Z', 'markets': [{'key': 'h2h', 'last_update': '2025-08-02T13:18:13Z', 'outcomes': [{'name': 'Hartberg', 'price': 2.4}, {'name': 'WSG Tirol', 'price': 2.5}, {'name': 'Draw', 'price': 2.8}]}]}, {'key': 'boylesports', 'title': 'BoyleSports', 'last_update': '2025-08-02T13:17:07Z', 'markets': [{'key': 'h2h', 'last_update': '2025-08-02T13:17:07Z', 'outcomes': [{'name': 'Hartberg', 'price': 2.6}, {'name': 'WSG Tirol', 'price': 2.4}, {'name': 'Draw', 'price': 3.1}]}]}, {'key': 'smarkets', 'title': 'Smarkets', 'last_update': '2025-08-02T13:18:14Z', 'markets': [{'key': 'h2h', 'last_u

In [ ]:
import Betting_Engines.H2hEventAboveMeanOddsBettingEngine as BettingEngine
import sys

from discord_bot import send_message

importlib.reload(BettingEngine)

bettingEngine = BettingEngine.H2hEventAboveMeanOddsBettingEngine(
    alpha=0.04,
    betOddsUpperLimit=2.5,
    betAmount=10,
    commision=0.0,
    singleBookmaker="betfair"
)

current_utc_time = datetime.now(timezone.utc).strftime("%Y-%m-%d_%H-%M-%S")

result_df = pd.DataFrame(bettingEngine.analyseAndFindBets(oddsApiEventsToH2hEvent(soccer_events)))
csv_url = f'Results/{current_utc_time}_results.csv'
result_df.to_csv(csv_url, index=False) 

if(len(result_df) > 0):
    send_message("Results:", csv_url)

print(result_df)

Empty DataFrame
Columns: []
Index: []
